In [1]:
import pandas as pd
import json
import functools
import re
import spacy
from spacy.tokens import Doc
import itertools as it
from nltk.tokenize import TweetTokenizer
from spacy.lang.en import STOP_WORDS
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import LdaModel

import pyLDAvis
import pyLDAvis.gensim
import pickle
import codecs
from spacy.pipeline import Pipe

E:\anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
'''
source: https://stackoverflow.com/questions/21708192/how-do-i-use-the-json-module-to-read-in-one-json-object-at-a-time/21709058
provided by: unutbu
'''
def json_parse(fileobj, decoder=json.JSONDecoder(), buffersize=2048, 
               delimiters=None):
    remainder = ''
    for chunk in iter(functools.partial(fileobj.read, buffersize), ''):
        remainder += chunk
        while remainder:
            try:
                stripped = remainder.strip(delimiters)
                result, index = decoder.raw_decode(stripped)
                yield result
                remainder = stripped[index:]
            except ValueError:
                # Not enough data to decode, read more
                break

In [3]:
with open('../data/tweets.txt', 'r') as rf:
    json_list = list(json_parse(rf))

In [4]:
len(json_list)

2000

In [5]:
for i in range(30, 35):
    json = json_list[i]
    try:
        text = json["retweeted_status"]["extended_tweet"]["full_text"]
    except KeyError:
        text = json["text"]
    print(text)
    print()

12652.916 USD  https://t.co/3T5IP5YsEn | #btc #bitcoin #blockchain #cryptocurrency https://t.co/eB0Ji7u78N

Something to consider if you suffer from #fomo over bitcoin: you might make or lose money by investing now, but you won’t lose anything by NOT investing. 

Few great quotes in this article: https://t.co/S910SsTFDX

12660.50 USD  https://t.co/vmknxeCRiL | #btc #bitcoin #blockchain #cryptocurrency https://t.co/a186DDXLAU

Lightning Network May Not Solve Bitcoin's Scaling 'Trilemma' https://t.co/umtLQyur31 #bitcoin #blockchain Via… https://t.co/tCpYmvbDJL

MAKE MONEY WITH bitcoin!!!

✅KUCOIN is 100% OPEN!!!

CLICK ▶️ https://t.co/lPgWGdMR6i

❌Binance 
❌Poloniex 
❌Bittre… https://t.co/YjUgq2T3mC



In [6]:
nlp = spacy.load('en')

E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode o

E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode o

E:\anaconda\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is depr

E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
E:\anaconda\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
E:\anaconda\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode o

In [7]:
ttkzr = TweetTokenizer(strip_handles=True, reduce_len=True)

In [8]:
docs = []
for json in json_list:
    try:
        text = json["retweeted_status"]["extended_tweet"]["full_text"]
    except KeyError:
        text = json["text"]
    text_nourl = re.sub(r"http\S+", "", text)
    tokens = ttkzr.tokenize(text_nourl)
    doc = Doc(nlp.vocab, words=tokens)
    docs.append(doc)

In [9]:
docs[90]

JOIN THE LARGEST #BITCOIN #CRYPTO NETWORK ON EARTH ! EMAIL : cryptopennystock@gmail.com #CryptoCurrencyGod #TEAMBILLIONAIRE #PENNYSTOCKS #STOCKALERT #BTC , #ETH , #ADA #XRP #TRX #LTC #BLOCKCHAIN 

In [10]:
docs_filepath = '../data/docs.txt'

In [11]:
with open(docs_filepath, 'w', encoding='utf_8') as wf:
    for doc in docs:
        wf.write(doc.text)
        wf.write("\n")

In [12]:
'''
source: Modern NLP in Python
author: Patrick Harrison
'''
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [13]:
unigram_sentences_filepath = '../data/unigram_sentences.txt'

In [14]:
with open(unigram_sentences_filepath, 'w', encoding='utf-8') as f:
    for sentence in lemmatized_sentence_corpus(docs_filepath):
        f.write(sentence + '\n')

In [15]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [16]:
for sentence in it.islice(unigram_sentences, 230, 240):
    print(sentence)

['join', '-PRON-', 'ico', 'now']
['here', 'be', 'a', 'link', 'and', '-PRON-', 'know', 'what', 'to', 'do', 'next', '😉']
['become', 'a', 'part', 'of', 'first', 'ecocryptomin', 'in', 'the', 'world']
['invest', 'investment', 'innovate', 'innovation', 'mining', 'miningfarm', 'crypto', 'profit', 'bitcoin', 'ethereum', 'cryptocurrency']
['st']
['james', 'place', 'condos', 'in', 'columbia', 'heights', 'accept', 'bitcoin', 'as', 'payment', 'washington', 'business', 'journal']
['rt']
['-PRON-', 'wish', 'man', 'would', 'put', 'as', 'much', 'confidence', 'in', 'woman', 'as', '-PRON-', 'do', 'in', 'bitcoin']
['there', 'be', 'a', 'direct', 'correlation', 'between', 'the', 'growth', 'in', 'the', 'number', 'of', 'successful', 'and', 'reliable', 'project', 'and', 'the', 'arrival', 'of', 'new', 'investor', 'on', 'the', 'market', 'whose', 'interest', 'will', 'increase', 'significantly']
['rr', 'revizorcoin', 'cryptocurrency', 'ico', 'blockchain']


In [17]:
crypto_names = pd.read_csv('../data/crypto-markets.csv')

In [18]:
crypto_names = crypto_names[['slug', 'symbol']]
crypto_names.drop_duplicates(inplace=True)

In [19]:
crypto_names['symbol'] = crypto_names['symbol'].str.lower()
crypto_names['slug'] = crypto_names['slug'].str.replace('-', '_')
crypto_names.head()

slug symbol
0          bitcoin    btc
1719      ethereum    eth
2607        ripple    xrp
4228  bitcoin_cash    bch
4400       cardano    ada

In [21]:
crypto_dict = dict(zip(crypto_names['symbol'].values, crypto_names['slug'].values))

In [22]:
crypto_dict

{'btc': 'bitcoin',
 'eth': 'ethereum',
 'xrp': 'ripple',
 'bch': 'bitcoin_cash',
 'ada': 'cardano',
 'ltc': 'litecoin',
 'xem': 'nem',
 'miota': 'iota',
 'xlm': 'stellar',
 'dash': 'dash',
 'neo': 'neo',
 'trx': 'tron',
 'eos': 'eos',
 'xmr': 'monero',
 'icx': 'icon',
 'btg': 'bitgem',
 'xrb': 'raiblocks',
 'qtum': 'qtum',
 'etc': 'ethereum_classic',
 'lsk': 'lisk',
 'omg': 'omisego',
 'xvg': 'verge',
 'sc': 'siacoin',
 'zec': 'zcash',
 'bnb': 'binance_coin',
 'bcn': 'bytecoin_bcn',
 'ppt': 'populous',
 'bcc': 'bitconnect',
 'ven': 'vechain',
 'strat': 'stratis',
 'kcs': 'kucoin_shares',
 'bts': 'bitshares',
 'usdt': 'tether',
 'ardr': 'ardor',
 'snt': 'status',
 'doge': 'dogecoin',
 'waves': 'waves',
 'steem': 'steem',
 'drgn': 'dragonchain',
 'dcn': 'dentacoin',
 'zrx': '0x',
 'dgb': 'digibyte',
 'rep': 'augur',
 'wax': 'wax',
 'veri': 'veritaseum',
 'ark': 'ark',
 'dent': 'dent',
 'etn': 'electroneum',
 'kmd': 'komodo',
 'hsr': 'hshare',
 'gnt': 'golem_network_tokens',
 'bat': 'batc

In [32]:
# Replace all symbols unigram_sentences with their more common name, using crypto_dict
unigram_sentences_replaced = []
for i, sentence in enumerate(unigram_sentences):
    for j, word in enumerate(sentence):
        key = word
        if key in crypto_dict:
            sentence[j] = crypto_dict[key]
    unigram_sentences_replaced.append(sentence)

In [37]:
unigram_sentences = unigram_sentences_replaced

In [41]:
sentences_nostop = []
STOP_WORDS.add("rt")
for i, sentence in enumerate(unigram_sentences):
    sentence_nostop = [token for token in sentence if token not in STOP_WORDS]
    sentences_nostop.append(sentence_nostop)

In [42]:
unigram_sentences = sentences_nostop

In [44]:
for sentence in it.islice(unigram_sentences, 230, 240):
    print(sentence)

['join', '-PRON-', 'ico']
['chainlink', '-PRON-', 'know', '😉']
['particl', 'ecocryptomin', 'world']
['invest', 'investment', 'innovate', 'innovation', 'mining', 'miningfarm', 'crypto', 'profit', 'bitcoin', 'ethereum', 'cryptocurrency']
['st']
['james', 'place', 'condos', 'columbia', 'heights', 'accept', 'bitcoin', 'payment', 'washington', 'business', 'journal']
[]
['-PRON-', 'mywish', 'man', 'putincoin', 'confidence', 'woman', '-PRON-', 'bitcoin']
['direct', 'correlation', 'growth', 'number', 'successful', 'reliable', 'project', 'arrival', 'new', 'investor', 'market', 'interest', 'increase', 'significantly']
['rr', 'revizorcoin', 'cryptocurrency', 'ico', 'blockchain']


In [45]:
bigram_sentences_filepath = '../data/bigram_sentences.txt'

In [46]:
# Trains the detector
bigram_model = Phrases(unigram_sentences)

E:\anaconda\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
E:\anaconda\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
E:\anaconda\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
E:\anaconda\lib\site-packages\gensim\models\phrases.py:317: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameter

In [47]:
with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = bigram_model[unigram_sentence]
        for word in bigram_sentence:
            f.write(word)
            f.write(" ")
        f.write("\n")

E:\anaconda\lib\site-packages\gensim\models\phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [48]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [52]:
for sentence in it.islice(bigram_sentences, 800, 850):
    print(sentence)

['ethereum', 'cloudmin', 'bitcoininvest']
['-PRON-', 'bitcoin', 'early_god', 'bless', '-PRON-', 'small_fortune']
['-PRON-', 'want_help']
['-PRON-', 'away_20', 'bitcoin', '20_random', 'people_retweet', 'jan_30th']
['-PRON-', 'hope_help', '-PRON-', '-PRON-', 'help', '-PRON-']
['$', '5,000', 'cryptocurrency', 'giveaway_nebulas_token', 'daily_ben', 'yu']
['alphabitcoinfund', 'news', '-PRON-']
['find', '-PRON-', 'visit']
['sudangc', 'sudan', 'gold', 'coin', 'moneycoin', 'blockchain', 'ethereum', 'ethereumclassic', 'bitcoin', 'ico', 'swaptoken', 'investment', 'cryptocurrency']
['news', 'yahoo', 'nasdaq', 'finance', 'business']
['abcnews']
['famous']
['$', '5', 'homeless', 'person', '-PRON-', 'buy', 'food', 'instead', 'invest', '-PRON-', 'bitcoin', 'smh']
['cindicator', 'cindicator', '-PRON-', '150']
['tech', 'datum', 'poly_ai', 'ml', 'bitcoin']
['ethereum']
['content_and_ad_network', '-PRON-', 'invite_allion', '-PRON-', 'girlfriend_come', 'vegas_force', 'cryptoball']
['-PRON-', 'allion', 'bi

In [53]:
dictionary_filepath = '../data/dictionary.dict'

In [54]:
dictionary = Dictionary(bigram_sentences)

In [65]:
dictionary.filter_extremes(no_below=5, no_above=0.5)
dictionary.compactify()
dictionary.save(dictionary_filepath)

In [60]:
def bow_generator(filepath):    
    for tweet in LineSentence(filepath):
        yield dictionary.doc2bow(tweet)

In [61]:
bow_filepath = "../data/bow.mm"

In [62]:
MmCorpus.serialize(bow_filepath, bow_generator(bigram_sentences_filepath))
bow_corpus = MmCorpus(bow_filepath)

In [63]:
bow_corpus[43]

[(6, 1.0), (14, 1.0), (30, 3.0), (40, 2.0), (61, 1.0)]